# ShapeFunction

In [1]:
using JFEM: ShapeFunction, Quad4

shape = ShapeFunction(Quad4)

# define local coordinate
ξ = [0.0,0.0]
    
@show N = shape(ξ)
@show ∂N∂ξ = shape(Val{:grad}, ξ)

# define geometry x of element
x = [[0.0,0.0], [1.0,0.0], [1.0,1.0], [0.0,1.0]]

@show ∂N∂x = shape(Val{:grad}, ξ, x)
@show J = shape(Val{:jacobian}, ξ, x) # Jacobian matrix ∂x/∂ξ
@show detJ = shape(Val{:detj}, ξ, x)  # determinant of Jacobian matrix

N = shape(ξ) = [0.25 0.25 0.25 0.25]
∂N∂ξ = shape(Val{:grad},ξ) = [-0.25 0.25 0.25 -0.25; -0.25 -0.25 0.25 0.25]
∂N∂x = shape(Val{:grad},ξ,x) = [-0.5 0.5 0.5 -0.5; -0.5 -0.5 0.5 0.5]
J = shape(Val{:jacobian},ξ,x) = [0.5 0.0; 0.0 0.5]
detJ = shape(Val{:detj},ξ,x) = 0.25


0.25

# Element

## 基本的な使い方

現在サポートしている要素の種類は，

* 1次元: `Bar2`, `Bar3`
* 2次元: `Quad4`, `Quad8`
* 3次元: `Hex8`

である．数字は要素の節点数を表す．各要素を作るには`connectivity`を引数として渡す必要がある．

In [2]:
using JFEM: Quad4, get_connectivity, get_dimension, get_gauss_points, get_shape_function

element = Quad4([1,2,3,4])
get_connectivity(element) # [1,2,3,4]
get_dimension(element) # 2
length(element) # 4
shape = get_shape_function(element)
gausspoints = get_gauss_points(element)

4-element Array{JFEM.GaussPoint,1}:
 [-0.57735,-0.57735]
 [0.57735,-0.57735] 
 [-0.57735,0.57735] 
 [0.57735,0.57735]  

### 変数の格納

要素の節点やガウスポイントに変数を保存するには`Field`を用いる．

In [3]:
using JFEM: Field

element["water head"] = Field([0.0,0.0,0.0,0.0]) # store node value (length must be 4)
push!(element["water head"], [1.0,1.0,0.0,0.0])
for gp in gausspoints
    gp["stress"] = Field([10.0,10.0,10.0])
end

### 形状関数$N_i$とその勾配$\partial{N_i}/\partial{X_j}, \partial{N_i}/\partial{x_j}$

In [4]:
## shape function (N) at a local coordinate ξ
element(ξ) # shape(ξ)

## To get a gradient of shape function, "geometry" must be stored in element beforehand.
element["geometry"] = Field([[0.0,0.0], [1.0,0.0], [1.0,1.0], [0.0,1.0]]) # set node coordinates (length must be 4)
push!(element["geometry"], [[0.0,0.0], [4.0,0.0], [4.0,4.0], [0.0,4.0]])

# gradient of shape function based on "the undeformed configuration (X)" (∂N/∂X)
@show element(Val{:GRAD}, ξ) # shape(Val{:grad}, ξ, element["geometry"][1])

# gradient of shape function based on "the current configuration (x)" (∂N/∂x)
@show element(Val{:grad}, ξ) # shape(Val{:grad}, ξ, element["geometry"][end])

element(Val{:GRAD},ξ) = [-0.5 0.5 0.5 -0.5; -0.5 -0.5 0.5 0.5]
element(Val{:grad},ξ) = [-0.125 0.125 0.125 -0.125; -0.125 -0.125 0.125 0.125]


2×4 Array{Float64,2}:
 -0.125   0.125  0.125  -0.125
 -0.125  -0.125  0.125   0.125

### $\sum{N_ih_i}$とその勾配$\sum{\frac{\partial{N}_i}{\partial{X}_j}h_i}, \sum{\frac{\partial{N}_i}{\partial{x}_j}h_i}$

In [5]:
## ∑Nᵢhᵢ

# element(fieldname, ξ; time=:end)
@show element("water head", ξ)

# corresponding method
N = element(ξ)
@show (N * element["water head"][end])[1]


## ∑∂Nᵢ/∂Xⱼ hᵢ

# element(fieldname, Val{:GRAD}, ξ; time=:end)
@show element("water head", Val{:GRAD}, ξ)

# corresponding method
∂N∂X = element(Val{:GRAD}, ξ)
@show ∂N∂X * element["water head"][end]


## ∑∂Nᵢ/∂xⱼ hᵢ

# element(fieldname, Val{:grad}, ξ; time=:end)
@show element("water head", Val{:grad}, ξ)

# corresponding method
∂N∂x = element(Val{:grad}, ξ)
@show ∂N∂x * element["water head"][end]

element("water head",ξ) = 0.5
(N * (element["water head"])[end])[1] = 0.5
element("water head",Val{:GRAD},ξ) = [0.0; -1.0]
∂N∂X * (element["water head"])[end] = [0.0,-1.0]
element("water head",Val{:grad},ξ) = [0.0; -0.25]
∂N∂x * (element["water head"])[end] = [0.0,-0.25]


2-element Array{Float64,1}:
  0.0 
 -0.25

### Jacobian matrix

In [6]:
# use reference configuration
@show element(Val{:JACOBIAN}, ξ)
@show element(Val{:detJ}, ξ)

# use current configuration
@show element(Val{:jacobian}, ξ)
@show element(Val{:detj}, ξ)

element(Val{:JACOBIAN},ξ) = [0.5 0.0; 0.0 0.5]
element(Val{:detJ},ξ) = 0.25
element(Val{:jacobian},ξ) = [2.0 0.0; 0.0 2.0]
element(Val{:detj},ξ) = 4.0


4.0

### 応用

In [7]:
δ = 0.1
element["geometry"] = Field([[0.0,0.0], [1.0,0.0], [1.0,1.0], [0.0,1.0]])
push!(element["geometry"], [[0.0,0.0], [1+δ,0.0], [1+δ,1-δ], [0.0,1-δ]])

F = [1.0+δ 0.0;
     0.0   1.0-δ]
F⁻¹ = inv(F)
I = eye(2)

# infinitesimal strain
∂u∂X = element("geometry", Val{:GRAD}, [0.0,0.0]; time=:dend)
@show ϵ = (∂u∂X + ∂u∂X') / 2

# Lagrange-Green strain
E = (∂u∂X + ∂u∂X' + ∂u∂X'*∂u∂X) / 2
@show E ≈ (F'*F - I)/2

# Euler-Almansi strain
∂u∂x = element("geometry", Val{:grad}, [0.0,0.0]; time=:dend)
e = (∂u∂x + ∂u∂x' - ∂u∂x'*∂u∂x) / 2
@show e ≈ (I - F⁻¹'*F⁻¹)/2

ϵ = (∂u∂X + ∂u∂X') / 2 = [0.1 0.0; 0.0 -0.1]
E ≈ (F' * F - I) / 2 = true
e ≈ (I - F⁻¹' * F⁻¹) / 2 = true


true